# TP 2 : Branch-and-bound applied to a knapsack problem

### Initialisation (à faire une seule fois)

In [1]:
import Pkg; 
Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


   Installed IrrationalConstants ───────── v0.1.1


   Installed ColorTypes ────────────────── v0.11.4


   Installed Adapt ─────────────────────── v3.4.0
   Installed GeometryTypes ─────────────── v0.8.5
   Installed OffsetArrays ──────────────── v1.12.8
   Installed TensorCore ────────────────── v0.1.1
   Installed IterTools ─────────────────── v1.4.0
   Installed ArnoldiMethod ─────────────── v0.2.0
   Installed Preferences ───────────────── v1.3.0
   Installed TableTraits ───────────────── v1.0.1


...

  ✓ GR_jll


  ✓ HTTP


  ✓ RecipesPipeline


  ✓ PlotThemes


  ✓ GR


### Récupération des données

In [1]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Tests de sondabilités TA, TO et TR basés sur le modèle linéaire

In [15]:
function TestsSondabilite_LP(model2, x, BestProfit, Bestsol)
    TA, TO, TR = false, false, false
    if(termination_status(model2) == MOI.INFEASIBLE)#Test de faisabilite
        TA=true
        println("TA")
    elseif(objective_value(model2) <= BestProfit) #Test d'optimalite
        TO=true
        println("TO")
    elseif( prod(abs.([round.(v, digits=0) for v in value.(x)]-value.(x)) .<= fill(10^-5, size(x))) 
        ) #Test de resolution
        TR=true
        println("TR")
        #if (value(benef) >= BestProfit)
        if (objective_value(model2) >= BestProfit)
            Bestsol = value.(x)
            #BestProfit=value(benef)
            BestProfit=objective_value(model2)
            println("\nNew Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")
        end
    else
        println("non sondable")
    end
    TA, TO, TR, Bestsol, BestProfit
end

function TestsSondabilite_Sac_A_Dos(listobjs, listval, placeDispo, profit, upperBound, lowerBound)
    TA, TO, TR = false, false, false
    if (placeDispo < 0)
        TA = true        
        println("TA")
    elseif (upperBound <= BestProfit)
        TO = true
        println("TO")
    elseif (upperBound == lowerBound)
        TR = true
        println("TR")
    else
        println("non sondable")
    end
    return TA, TO, TR
end


TestsSondabilite_Sac_A_Dos (generic function with 1 method)

## Procédure de séparation (branching) et stratégie d'exploration permettant de se placer au prochain noeud à traiter

In [22]:

function SeparerNoeud_lexicographic_depthfirst!(listobjs, listvals, n)
    # this node is non-sondable. Apply the branching criterion to separate it into two subnodes
    # and choose the child-node at the left  
    
    # lexicographic branching criterion: branch on the 1st object not yet fixed
    i, obj = 1, 0
    while((i <= n) && (obj==0))
        if(!(i in listobjs))
            obj=i
        end
        i+=1
    end
    
    println("\nbranch on object ", obj, "\n")

    # depthfirst exploration strategy: the node selected will be the most left of the child-nodes just created
    push!(listobjs,obj) #save the identity of the object selected for branching
    push!(listvals,1.0) #save the node selected, identified by the value assigned to the variable/object chosen
end

function get_Meilleur_Coeff(price,weight,listobjs)
    n = length(price)
    i, obj, maxr = 1, 0, -1.0
    while((i <= n))
        if (!(i in listobjs)) && ((price[i]/weight[i]) >= maxr)
            maxr = (price[i]/weight[i])
            obj = i
        end        
        i+=1
    end
    return obj
end

function SeparerNoeud_Plus_efficace(listobjs, listvals, price, weight)
    # Cette fonction est la fonction de séparation indiquant dans son retour quel est l'objet choisi à ce noeud
    # à partir du coefficient d'efficacité gain/coût contrainte

    obj = get_Meilleur_Coeff(price,weight,listobjs)
    println("\nbranch on object ", obj, "\n")
    push!(listobjs,obj) #save the identity of the object selected for branching
end

function ExplorerAutreNoeud_depthfirst!(listobjs, listvals, listnodes, borne)
    #this node is sondable, go back to parent node then right child if possible
    
    stop=false
    #check if we are not at the root node
    if (length(listobjs)>= 1)
        #go back to parent node
        obj=pop!(listobjs)
        theval=pop!(listvals)
        tmp=pop!(listnodes)

        #go to right child if possible, otherwise go back to parent
        while( (theval==0.0) && (length(listobjs)>= 1))
            obj=pop!(listobjs)
            theval=pop!(listvals)
            tmp=pop!(listnodes)
        end
        if theval==1.0
            push!(listobjs,obj)
            push!(listvals,0.0)
        else
            println("\nFINISHED")
            stop=true
        end
    else
        #the root node was sondable
        println("\nFINISHED")
        stop=true
    end
    return stop 
end

function ExplorerNoeud(listobjs, listvals, listnodes, price, weight, profitEnCours, placeDispo, BestProfit, choixBorne)

    stop = false

    objectselect = pop!(listobjs)
    push!(listobjs,objectselect)

    println("\nTest val = 1")

    push!(listvals,1.0)

    if choixBorne == 1
        LB1,UB1 = Bornes_1!(price, weight, listobjs, placeDispo - weight[objectselect], profitEnCours+price[objectselect])
    elseif choixBorne == 2
        LB1,UB1 = Bornes_2!(price, weight, listobjs, placeDispo - weight[objectselect], profitEnCours+price[objectselect])
    else
        error("Methode de borne non defini (n'accepte que 1 ou 2)")
    end

    TA1,TO1,TR1,BestProfit1 = TestsSondabilite_Sac_A_Dos(listobjs, listvals, placeDispo - weight[objectselect], profitEnCours+price[objectselect], UB1, LB1)
    is_node_sondable1 = TA1 || TO1 || TR1

    pop!(listvals)


    println("\nTest val = 0")

    push!(listvals,0.0)

    if choixBorne == 1
        LB1,UB1 = Bornes_1!(price, weight, listobjs, placeDispo, profitEnCours)
    elseif choixBorne == 2
        LB1,UB1 = Bornes_2!(price, weight, listobjs, placeDispo, profitEnCours)
    else
        error("Methode de borne non defini (n'accepte que 1 ou 2)")
    end

    TA0,TO0,TR0,BestProfit0 = TestsSondabilite_Sac_A_Dos(listobjs, listvals, placeDispo, profitEnCours, BestProfit, UB1, LB1, length(price))
    is_node_sondable0 = TA0 || TO0 || TR0
    
    pop!(listvals)


    if !(is_node_sondable1) && ((is_node_sondable0) || UB1 >= UB0)
        push!(listvals,1.0)
        profitEnCours += price[objectselect]
        placeDispo -= weight[objectselect]
        BestProfit = BestProfit1
        println("\nVal = 1")
    
    elseif !(is_node_sondable0) && ((is_node_sondable1) || UB1 < UB0)
        push!(listvals,0.0)
        BestProfit = BestProfit0        
        println("\nVal = 0")

    else  
        println("\nFINISHED")
        stop=true
    end

    if (profitEnCours >= BestProfit)
        BestProfit = profitEnCours
    end

    return stop, listobjs, listvals, listnodes, profitEnCours, placeDispo, BestProfit
end


function Affichage_sol(n,listobjs,listvals)
    sol = zeros(n)
    listobjsCopy=copy(listobjs)
    listvalsCopy=copy(listvals)
    while !(isempty(listobjsCopy) || isempty(listvalsCopy))
        obj = pop!(listobjsCopy)
        val = pop!(listvalsCopy)
        sol[obj] = val
    end
    return sol
end
    
    

Affichage_sol (generic function with 1 method)

###  Création de la relaxation linéaire (= modèle associé au noeud 0): <span style="color:red"> SECTION A SUPPRIMER !!!! </span>

<span style="color:red"> Cette section est à commenter/supprimer et remplacer par vos propres calculs de bornes supérieures et autres, par exemple basées sur les bornes 1 et 2 vues en cours, ou d'autres calculs de bornes de votre choix/conception validés au préalable par votre encadrant/e de TP </span>

In [23]:
#=Pkg.add("Clp");
Pkg.add("JuMP");
using JuMP, Clp

function MajModele_LP!(model2, x, listobjs, listvals)
    for i in 1:length(listobjs)
        set_lower_bound(x[listobjs[i]],listvals[i])
        set_upper_bound(x[listobjs[i]],listvals[i])
    end
end

function Reset_LP!(model2, x, listobjs)
    for i in 1:length(listobjs)
        set_lower_bound(x[listobjs[i]],0.0)
        set_upper_bound(x[listobjs[i]],1.0)
    end
end

function Reset_allLP!(model2, x)
    for i in 1:length(x)
        set_lower_bound(x[i],0.0)
        set_upper_bound(x[i],1.0)
    end
end

function CreationModele_LP(price, weight, capacity),
# ROOT NODE
    
    n=length(price)
    
    model2 = Model(Clp.Optimizer) # set optimizer
    set_optimizer_attribute(model2, "LogLevel", 0) #don't display anything during solve
    set_optimizer_attribute(model2, "Algorithm", 4) #LP solver chosen is simplex

    # define x variables as CONTINUOUS (recall that it is not possible to define binary variables in Clp)
    @variable(model2, 0 <= x[i in 1:n] <= 1)

    # define objective function
    @objective(model2, Max, sum(price[i]*x[i] for i in 1:n))

    # define the capacity constraint 
    @constraint(model2, sum(weight[i]*x[i] for i in 1:n) <=  capacity)

    println(model2) 

    return model2, x
end=#


function Bornes_1!(price, weight, listobjs, placeDispo, profitEnCours)
    if (length(price)==length(listobjs))
        ajout = 0
    else
    # Recup de l'objet choisi par la séparation
        obj = get_Meilleur_Coeff(price,weight,listobjs)
        ajout = price[obj]*placeDispo/weight[obj]
    end
    

    return profitEnCours, (profitEnCours+ajout)
end

function Bornes_2!(price, weight, listobjs, placeDispo, profitEnCours)
    
    prixMax, listeAjoutee = profitEnCours, copy(listobjs)

    # Recup de l'objet choisi par la séparation
    objselect = pop!(listobjs)   
    push!(listobjs,objselect)
    placeDispo -= weight[objselect]
    if (length(price)==length(listobjs))
        ajout = 0
    else
        
        # Ajout d'une fois les plus efficaces jusqu'à ce que l'on peut plus
        while (true)
            placeDispoObtenue = placeDispo
            obj = get_Meilleur_Coeff(price,weight,listeAjoutee)
            
            placeDispoObtenue -= weight[obj]
            if (placeDispoObtenue < 0)
                break
            end
            push!(listeAjoutee,obj)
            placeDispo = placeDispoObtenue
            prixMax += price[obj]
        end
        # Ajout de la fraction permettant d'avoir la capacité maximale
        obj = get_Meilleur_Coeff(price,weight,listeAjoutee)
        
        ajout = price[obj]*placeDispo/weight[obj]
    end

    return profitEnCours+price[objselect],prixMax + ajout + price[objselect]
end


Bornes_2! (generic function with 1 method)

### Boucle principale : résoudre une relaxation, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [24]:

function SolveKnapInstance(filename)

    price, weight, capacity = readKnaptxtInstance(filename)

    #model2, x = CreationModele_LP(price, weight, capacity)

    #create the structure to memorize the search tree for visualization at the end
    trParentnodes=Int64[] #will store orig node of arc in search tree
    trChildnodes=Int64[] #will store destination node of arc in search tree
    trNamenodes=[] #will store names of nodes in search tree

    #intermediate structure to navigate in the search tree
    listobjs=[]
    listvals=[]
    listnodes=[]

    BestProfit=-1
    Bestsol=[]

    borne = 2

    current_node_number=0
    placeDispo, profitEnCours = capacity, 0
    stop = false

    while(!stop)

        println("\nNode number ", current_node_number, ": \n---------------\n")

        #Update the graphical tree
        push!(trNamenodes,current_node_number+1) 
        if(length(trNamenodes)>=2)
            push!(trParentnodes,listnodes[end]+1) # +1 because the 1st node is "node 0"
            push!(trChildnodes, current_node_number+1) # +1 because the 1st node is "node 0"
        end
        push!(listnodes, current_node_number)

        #=
        #create LP of current node
        MajModele_LP!(model2, x, listobjs, listvals)
        
        println(model2)
        
        print("Solve the LP model of the current node to compute its bound: start ... ")

        status = optimize!(model2)

        println("... end"); 

        print(": Solution LP")
        if(termination_status(model2) == MOI.INFEASIBLE)#(has_values(model2))
            print(" : NOT AVAILABLE (probably infeasible or ressources limit reached)")
        else
            print(" ", objective_value(model2))
            [print("\t", name(v),"=",value(v)) for v in all_variables(model2)] 
        end
        println(" "); 

        =#

        SeparerNoeud_Plus_efficace(listobjs, listvals,  price, weight)

        println("\nPrevious Solution memorized ", Bestsol, " with bestprofit ", BestProfit, "\n")
        #=
        TA, TO, TR, Bestsol, BestProfit = TestsSondabilite_LP(model2, x, BestProfit, Bestsol)

        is_node_sondable = TA || TO || TR

        #Reset_LP!(model2, x, listobjs)

        if(!is_node_sondable)
            SeparerNoeud_lexicographic_depthfirst!(listobjs, listvals, length(price))
        else
            stop = ExplorerAutreNoeud_depthfirst!(listobjs, listvals, listnodes)
        end
        =#
        stop, listobjs, listvals, listnodes, profitEnCours, placeDispo, BestProfit = ExplorerNoeud(listobjs, listvals, listnodes, price, weight, profitEnCours, placeDispo, BestProfit, borne)
        Bestsol = Affichage_sol(length(price),listobjs,listvals)
        println("Place restante dans le sac ", placeDispo," pour une valeur de ", profitEnCours)
        #Reset_allLP!(model2, x)

        current_node_number = current_node_number + 1
    end

    println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)

    return BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes

end


SolveKnapInstance (generic function with 1 method)

### Affichage du résultat final

In [25]:
BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes = SolveKnapInstance("instancesETU/KNAPnewformat/test.opb.txt")
println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)
#graphplot(trParentnodes, trChildnodes, names=trNamenodes, method=:tree)


Node number 0: 
---------------


branch on object 2


Previous Solution memorized Any[] with bestprofit -1



UndefVarError: UndefVarError: Borne_2! not defined

In [26]:
BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes = SolveKnapInstance("instancesETU/KNAPnewformat/strongly_correlated/knapPI_3_50_1000_2_-1678.opb.txt")



Node number 0: 
---------------


branch on object 30


Previous Solution memorized Any[] with bestprofit -1



UndefVarError: UndefVarError: Borne_2! not defined

In [27]:
Questions préliminares :

1. La règle de séparation permet de divisier une situation en deux cas à observer. Dans le problème du sac, on observe les variations (présent ou pas présent) pour l'objet $i$ ayant le plus grand ratio, c'est à dire $\frac{p_i}{c_i}.
On sépare donc notre situation dans un premier cas où l'objet $i$ solution de $max = \frac{p_i}{c_i}$ est présent, et un second cas où il n'est pas présent.
SeparerNoeud_lexicographic_depthfirst!

2. Pour le calcul de la borne supérieure, on prend la valeur maximale $x_i$ dans R de la variable choisit dans l'étape de séparation qui permet de respecter la contrainte.


3. Le test d'admissibilité permet de vérifier que l'on respecte bien les contraintes. 
   Le test d'Optimalité permet de vérifier qu'un résultat réalisable est plus optimal qu'un autre résultat dejà calculé étant réalisable.
   Le test de résolution vérifie que la borne supérieur d'un état est égal à sa borne inférieur. Dans ce cas, cet état est la solution.

4. On choisi le cas tel le UpperBound est maximal.

ErrorException: syntax: extra token "préliminares" after end of expression